# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [1]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [2]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [4]:
from openai import OpenAI
from auth import api_key
client = OpenAI(api_key=api_key)
def get_llm_response(prompt, system_msg="You are a friendly assistant."):
    response = client.chat.completions.create(
        model="gpt-4o-mini",           # or any model you prefer
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

# TODO: Test your get_llm_response function with different prompts
prompt = "hi, how are you today"
print(get_llm_response(prompt))

Hello! I'm just a computer program, but I'm here and ready to help you. How can I assist you today?


## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [5]:
# List of healthcare questions to test
import pandas as pd
from itertools import product
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# TODO: Compare the different prompting strategies on these questions
prompt_templates = {
    "plain": "{q}",
    "cot": "{q}\n\nThink step-by-step.",
}

system_templates = {
    "plain": "You are a friendly assistant.",
    "role":  "You are an experienced clinician speaking to a layperson.",
    "cot":   "You are a friendly assistant.",
}
records = []

for q, strat in product(questions, prompt_templates):
    user_prompt = prompt_templates[strat].format(q=q)
    sys_prompt  = system_templates.get(strat, system_templates["plain"])
    
    answer = get_llm_response(user_prompt, system_msg=sys_prompt)
    
    records.append(
        {"question": q, "strategy": strat, "prompt": user_prompt,
         "system_message": sys_prompt, "answer": answer}
    )

df = pd.DataFrame(records)
df.to_csv("gout_prompting_results.csv", index=False)
print(df.head())

                                            question strategy  \
0  What foods should be avoided by patients with ...    plain   
1  What foods should be avoided by patients with ...      cot   
2  What medications are commonly prescribed for g...    plain   
3  What medications are commonly prescribed for g...      cot   
4                  How can gout flares be prevented?    plain   

                                              prompt  \
0  What foods should be avoided by patients with ...   
1  What foods should be avoided by patients with ...   
2  What medications are commonly prescribed for g...   
3  What medications are commonly prescribed for g...   
4                  How can gout flares be prevented?   

                  system_message  \
0  You are a friendly assistant.   
1  You are a friendly assistant.   
2  You are a friendly assistant.   
3  You are a friendly assistant.   
4  You are a friendly assistant.   

                                              answer  


## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [7]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    response = response.lower()
    hits = sum(1 for kw in keywords if kw.lower() in response)
    return hits / len(keywords) if keywords else 0.0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
df["score"] = df.apply(
    lambda row: score_response(
        row["answer"],
        expected_keywords[row["question"]]
    ),
    axis=1
)
summary = (
    df.groupby("strategy")["score"]
      .mean()                       # mean across the 5 questions
      .sort_values(ascending=False) # best at the top
)
print("Average keyword-coverage score per strategy:")
print(summary)

best_strategy = summary.idxmax()
print(f"Best overall strategy: {best_strategy!r} "
      f"with mean score {summary[best_strategy]:.2%}")

Average keyword-coverage score per strategy:
strategy
cot      0.96
plain    0.96
Name: score, dtype: float64
Best overall strategy: 'cot' with mean score 96.00%


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [13]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy
from pathlib import Path

# ---------------------------------- CONFIG ----------------------------------
OUT_DIR   = Path("results/part_3")
OUT_FILE  = OUT_DIR / "prompting_results.txt"

strategy_labels = {          # human-friendly headings
    "plain": "Zero-shot",
    "role":  "One-shot",
    "cot":   "Few-shot",
}

OUT_DIR.mkdir(parents=True, exist_ok=True)   

lines = ["# Prompt Engineering Results\n"]
for q in questions:
    lines.append(f"## Question: {q}\n")
    for strat in df["strategy"].unique():              # preserves encounter order
        pretty = strategy_labels.get(strat, strat.title())
        resp = df.loc[(df.question == q) & (df.strategy == strat),
                      "answer"].iloc[0]
        lines.append(f"### {pretty} response:\n{resp}\n")
    lines.append("-" * 50 + "\n")
pivot = (
    df.pivot_table(index="question", columns="strategy", values="score")
      .reindex(columns=df["strategy"].unique())        # keep column order
      .reset_index()
)
def slug(text: str) -> str:
    return text.lower().replace("?", "").replace(",", "") \
               .replace(" ", "_")

csv_lines = []

# header row
csv_lines.append(",".join(["question", *pivot.columns[1:]]))

# per-question rows
for _, row in pivot.iterrows():
    csv_lines.append(",".join([slug(row["question"]),
                               *row.iloc[1:].round(2).astype(str)]))

# averages row
avg_scores = df.groupby("strategy")["score"].mean().round(2)
csv_lines.append(",".join(["average", *avg_scores.astype(str)]))

# best-performing strategy row
best_method = avg_scores.idxmax()
csv_lines.append(",".join(["best_method", best_method]))

# wrap CSV block in fenced code for markdown
lines.append("## Scores\n\n```\n")
lines.append("\n".join(csv_lines) + "\n```\n")
OUT_FILE.write_text("\n".join(lines), encoding="utf-8")


22766

## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method